In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Difference-JSON").getOrCreate() #subtract: difference, intersection: common
print(spark.sparkContext.appName)

22/01/07 12:01:41 WARN Utils: Your hostname, Anishs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.90.85 instead (on interface en0)
22/01/07 12:01:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/07 12:01:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Difference-JSON


In [2]:
json1 = "/Users/anishchapagain/Downloads/vericred_fallon_20220103.json"
json2 = "/Users/anishchapagain/Downloads/vericred_fallon_20220105.json"
print([json1,json2])

['/Users/anishchapagain/Downloads/vericred_fallon_20220103.json', '/Users/anishchapagain/Downloads/vericred_fallon_20220105.json']


In [12]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.pandas as ps

In [4]:
df1 = spark.read.json(path=json1)

In [5]:
df1.count()

34701

In [6]:
df1.distinct().count()

34700

In [7]:
df2 = spark.read.json(path=json2)

In [8]:
df2.count()

38545

In [9]:
df2.distinct().count()

38545

In [13]:
data = [1,2,3,4]
df = ps.DataFrame(data, columns=["a"])

In [16]:
networks = df1.select(explode(df1.networks)).select('col.name','col.tier') #col: API name
networks.show(3) 

+--------------------+----+
|                name|tier|
+--------------------+----+
|         Direct Care|    |
|         Select Care|    |
|Fallon Preferred ...|    |
+--------------------+----+
only showing top 3 rows



In [23]:
netDF = networks.toPandas()
netDF.head()

,name,tier
0,Direct Care,
1,Select Care,
2,Fallon Preferred Care PPO,
3,Steward Community Care,
4,The City of Worcester Advantage - Direct Plan,


In [28]:
netDF.plot(kind=pie)

NameError: name 'pie' is not defined

In [24]:
type(netDF)

pandas.core.frame.DataFrame

In [18]:
df3=ps.DataFrame(networks)
df3.head()

22/01/07 15:52:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/01/07 15:52:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,name,tier
0,Direct Care,
1,Select Care,
2,Fallon Preferred Care PPO,
3,Steward Community Care,
4,The City of Worcester Advantage - Direct Plan,


In [21]:
df3[df3.name=="Fallon Preferred Care PPO"].count()

name    32107
tier    32107
dtype: int64

In [22]:
df3.info

22/01/07 15:55:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/01/07 15:55:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


<bound method DataFrame.info of                                                  name    tier
0                                         Direct Care        
1                                         Select Care        
2                           Fallon Preferred Care PPO        
3                              Steward Community Care        
4       The City of Worcester Advantage - Direct Plan        
5                                            NaviCare        
6                          Fallon Medicare Plus (HMO)        
7                                         Direct Care        
8                                         Select Care        
9                           Fallon Preferred Care PPO        
10                             Steward Community Care        
11                                     Community Care        
12      The City of Worcester Advantage - Direct Plan        
13                                           NaviCare        
14                         Fallon Medi

In [14]:
df

,a
0,1
1,2
2,3
3,4


In [10]:
#difference: same schema required!
sub = df2.subtract(df1.distinct())

AnalysisException: Except can only be performed on tables with the compatible column types. array<struct<address_string:string,city:string,languages:array<struct<name:string,type:string>>,office_name:string,phones:array<struct<type:string,value:string>>,state:string,street_line_1:string,street_line_2:string,zip:string>> <> array<struct<accepting_new_patients:boolean,address_string:string,city:string,languages:array<struct<name:string,type:string>>,office_name:string,pcp:boolean,pcp_id:string,phones:array<struct<type:string,value:string>>,state:string,street_line_1:string,street_line_2:string,zip:string>> at the first column of the second table;
'Except false
:- Relation [addresses#54,group_affiliations#55,hospital_affiliations#56,networks#57,provider#58,specialties#59] json
+- Deduplicate [specialties#12, networks#10, provider#11, group_affiliations#8, hospital_affiliations#9, addresses#7]
   +- Relation [addresses#7,group_affiliations#8,hospital_affiliations#9,networks#10,provider#11,

In [108]:
sub.count()

36852

In [110]:
newUnion

DataFrame[addresses: array<struct<accepting_new_patients:string,address_string:string,city:string,languages:array<struct<name:string,type:string>>,office_name:string,pcp:string,pcp_id:string,phones:array<struct<type:string,value:string>>,state:string,street_line_1:string,street_line_2:string,zip:string>>, group_affiliations: array<struct<name:string>>, hospital_affiliations: array<struct<name:string>>, networks: array<struct<name:string,tier:string>>, provider: struct<accepting_new_patients:boolean,facility_name:string,facility_type:string,first_name:string,gender:string,last_name:string,license_number:string,middle_name:string,npi:string,pcp:boolean,pcp_id:string,provider_type:string,rating:struct<scale:string,score:string>,site_uid:string,suffix:string,title:string,unparsed_name:string>, specialties: array<struct<name:string>>]

In [103]:
unionDF.head(3)

[Row(addresses=[Row(accepting_new_patients=None, address_string='1 Parker Ln, Perry, NY 14530', city='Perry', languages=[Row(name='English', type='primary'), Row(name='Persian (farsi)', type='secondary'), Row(name='Hindi', type='secondary'), Row(name='Urdu', type='secondary')], office_name='', pcp=None, pcp_id=None, phones=[Row(type='phone', value='(585) 969-5335')], state='NY', street_line_1='1 Parker Ln', street_line_2='', zip='14530')], group_affiliations=[Row(name='Wyoming County Community Hospital')], hospital_affiliations=[Row(name='Wyoming County Community Hospital (FULL)')], networks=[Row(name='Child Health Plus', tier='')], provider=Row(accepting_new_patients=True, facility_name=None, facility_type=None, first_name='Salman', gender='M', last_name='Abbasey', license_number='', middle_name='N', npi=None, pcp=None, pcp_id=None, provider_type='individual', rating=Row(scale=None, score=None), site_uid=None, suffix='MBBS', title=None, unparsed_name='Salman N Abbasey MBBS'), specialt

In [104]:
unionDF.tail(3)

[Row(addresses=[Row(accepting_new_patients=None, address_string='1425 Portland Ave Ste 228, Rochester, NY 146213001', city='Rochester', languages=[Row(name='English', type='primary')], office_name='', pcp=None, pcp_id=None, phones=[Row(type='phone', value='(585) 922-2575')], state='NY', street_line_1='1425 Portland Ave Ste 228', street_line_2='', zip='146213001'), Row(accepting_new_patients=None, address_string='1425 Portland Ave Ste 304, Rochester, NY 146213001', city='Rochester', languages=[Row(name='English', type='primary')], office_name='', pcp=None, pcp_id=None, phones=[Row(type='phone', value='(585) 922-5067')], state='NY', street_line_1='1425 Portland Ave Ste 304', street_line_2='', zip='146213001'), Row(accepting_new_patients=None, address_string='1425 Portland Ave Box 228, Rochester, NY 146213001', city='Rochester', languages=[Row(name='English', type='primary')], office_name='', pcp=None, pcp_id=None, phones=[Row(type='phone', value='(585) 922-4000')], state='NY', street_lin

In [114]:
def writeToFile(df,fName,opDir,mode):
    import os
    if not os.path.exists(opDir):
        os.makedirs(opDir)
        print("Directory Created")       
    
    if mode=='csv':
        df.write.mode('Overwrite').csv(opDir)    
    else:
        print("@@")
        df.write.mode('Overwrite').json(opDir)
        
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
    list_status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(opDir))
    print(list_status)
    file_name = [file.getPath().getName() for file in list_status if file.getPath().getName().startswith('part-')][0]
    print(file_name)
#     print("Renaming "+file_name+" .... "+fName)
    fs.rename(spark._jvm.org.apache.hadoop.fs.Path(opDir+'/'+file_name),spark._jvm.org.apache.hadoop.fs.Path(opDir+'/'+fName))
    for file_name in os.listdir(opDir): #remove part- files if they exist!
        if file_name.startswith('part-'):
            print(file_name)
#             os.remove(opDir+'/'+file_name)
# cat merged/part*.json > merged/final.json 

In [115]:
writeToFile(newUnion,'vericred_univera1.json','merged','json')

@@


[Lorg.apache.hadoop.fs.FileStatus;@7d3b748a
part-00000-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00010-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00008-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00005-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00012-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00007-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00002-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00003-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00006-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00013-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00004-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00009-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00011-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json
part-00001-8496fe1d-81c1-4edc-ac7d-f7ff90249f69-c000.json


In [121]:
cat merged/part*.json > merged/final.json 

In [12]:
# spark.stop()

In [ ]:
unionDf.write.json('filename.json')
unionDf.write.mode('Overwrite').json('merge')